In [7]:
import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pyomo.opt import SolverFactory
import os

In [8]:
# data
data = pd.read_excel('inputs.xlsx')

In [9]:
def solve_opt(data, pass_min=0, pass_max=5990, mult_1=1, mult_2=1, k=100, C_bat=80, alpha=10, beta=5, P_sell=0.25, tee=False):
    model = pyo.ConcreteModel() # create model
    # vary mult 1 to create pareto front!!!
    t = 96

    D_passenger = (data['Total demand']* mult_1).tolist()
    O_energy = (data['Outage']* mult_2).tolist()
    Riding_price = 0.5
    P_buy = 0.1

    ch_eff = 0.90
    dch_eff = 1/0.9
    gama = 1.5 # 30km/h
    E_0 = 0.2
    E_min = 0.2
    E_max = 1
    
    R_bat = 150
    Cycle = 3000
    DoD = 60
    T = 96
    delta_t = 0.25
    
    # sets
    model.T = pyo.RangeSet(t) # set of timesteps
    model.K = pyo.RangeSet(k) # set of SAEVs

    # parameters
    model.D_passenger = pyo.Param(model.T, initialize=lambda model, t: D_passenger[t-1]) # operational voltage of charger n
    model.O_energy = pyo.Param(model.T, initialize=lambda model, t: O_energy[t-1]) # operational voltage of charger n
    
    # binary variables
    model.b = pyo.Var(model.K, model.T, within=pyo.Binary) # binary variable indicating if bus k is serving trip i at time t
    model.x = pyo.Var(model.K, model.T, domain=pyo.Binary) # binary variable indicating if bus k is occupying a charger n at time t to charge
    model.y = pyo.Var(model.K, model.T, domain=pyo.Binary) # binary variable indicating if bus k is occupying a charger n at time t to discharge

    # non-negative variables
    model.e = pyo.Var(model.K, model.T, within=pyo.NonNegativeReals) # energy level of bus k at time t
    model.w_buy = pyo.Var(model.T, within=pyo.NonNegativeReals) # electricity purchased from the grid at time t
    model.w_sell = pyo.Var(model.T, within=pyo.NonNegativeReals) # electricity sold to the grid at time t
    model.w_riding = pyo.Var(model.T, within=pyo.NonNegativeReals) # number of trips performed at time t
    model.d = pyo.Var(model.T, within=pyo.NonNegativeReals) # total degradation cost of the bus k battery at time t

    # constraints
    model.constraints = pyo.ConstraintList()  # Create a set of constraints

    for k in model.K:
        for t in model.T:
            model.constraints.add(model.b[k,t] + model.x[k,t] + model.y[k,t] <=1)

    #for t in model.T:
        model.constraints.add(sum(model.b[k, t] for k in model.K) <= (model.D_passenger[t]))

    for t in model.T:
        model.constraints.add(sum(model.b[k, t] for k in model.K) == model.w_riding[t])
     
    # passengers   
    model.constraints.add(sum(model.w_riding[t] for t in model.T) >= pass_min)
    model.constraints.add(sum(model.w_riding[t] for t in model.T) <= pass_max)


    for k in model.K:
        for t in range(2,T+1):
            model.constraints.add(model.e[k,t] == model.e[k,t-1] + delta_t*ch_eff*alpha*model.x[k,t] - gama*model.b[k,t] - delta_t *dch_eff*beta*model.y[k,t])

    for t in model.T:
        model.constraints.add(delta_t * sum(ch_eff*alpha*model.x[k,t] for k in model.K) == model.w_buy[t])

    for t in model.T:
            model.constraints.add(delta_t * sum(dch_eff*beta*model.y[k,t] for k in model.K) == model.w_sell[t])

    for t in model.T:
        model.constraints.add(delta_t * sum(dch_eff*beta*model.y[k,t] for k in model.K) <= (delta_t * model.O_energy[t]))

    for k in model.K:
        for t in model.T:
            model.constraints.add(model.e[k,t] >= C_bat * E_min)

    for k in model.K:
        for t in model.T:
            model.constraints.add(E_max * C_bat >= model.e[k,t])          

    for k in model.K:
        model.constraints.add(model.e[k,1] == E_0*C_bat)

        for t in model.T:
            model.constraints.add(model.d[t] == ((R_bat)/(Cycle*DoD) * model.w_sell[t]))

    # objective function
    def rule_obj(mod):
        return sum(Riding_price*mod.w_riding[t] for t in mod.T) - sum(P_buy*mod.w_buy[t] for t in mod.T) + sum(P_sell*mod.w_sell[t] for t in mod.T) - sum(mod.d[t] for t in mod.T) 
    model.obj = pyo.Objective(rule=rule_obj, sense=pyo.maximize)
    
    # SOLVER
    opt = pyo.SolverFactory('gurobi')
    opt.options['timelimit'] = 600
    opt.options['mipgap'] = 0.01
    results = opt.solve(model, tee=tee)
    #print(results)
    return model

In [10]:
# Save function
def save_to_excel(model, filename='output.xlsx', Riding_price = 0.5, P_buy = 0.1, P_sell = 0.25, delta_t = 0.25):

    # Create a dictionary to store variable values
    var_values = {}

    # Store values of binary variables
    var_values['b'] = {(k, t): pyo.value(model.b[k, t]) for k in model.K for t in model.T}
    var_values['x'] = {(k, t): pyo.value(model.x[k, t]) for k in model.K for t in model.T}
    var_values['y'] = {(k, t): pyo.value(model.y[k, t]) for k in model.K for t in model.T}

    # Store values of non-negative variables
    var_values['e'] = {(k, t): pyo.value(model.e[k, t]) for k in model.K for t in model.T}
    var_values['w_buy'] = {t: pyo.value(model.w_buy[t]) for t in model.T}
    var_values['w_sell'] = {t: pyo.value(model.w_sell[t]) for t in model.T}
    var_values['w_riding'] = {t: pyo.value(model.w_riding[t]) for t in model.T}
    var_values['d'] = {(t): pyo.value(model.d[t]) for t in model.T}

    # Store objective function values
    total_revenues = pyo.value(sum(Riding_price * model.w_riding[t] for t in model.T) - sum(delta_t*P_buy * model.w_buy[t] for t in model.T) + sum(delta_t*P_sell * model.w_sell[t] for t in model.T) - sum(model.d[t] for t in model.T))
    total_passengers = pyo.value(sum(model.w_riding[t] for t in model.T))
    total_energy_provided = pyo.value(delta_t*sum(model.w_sell[t] for t in model.T))
    total_energy_charged = pyo.value(delta_t*sum(model.w_buy[t] for t in model.T))
    total_degradation = pyo.value(sum(model.d[t] for t in model.T))

    objective_values = {
        'total_revenues': total_revenues,
        'total_passengers': total_passengers,
        'total_energy_provided': total_energy_provided,
        'total_energy_charged': total_energy_charged,
        'total_degradation': total_degradation
    }

    # Convert dictionary to DataFrame and save to Excel
    with pd.ExcelWriter(filename) as writer:
        for var, values in var_values.items():
            df = pd.DataFrame.from_dict(values, orient='index', columns=[var])
            df.to_excel(writer, sheet_name=var)
        
        obj_df = pd.DataFrame.from_dict(objective_values, orient='index', columns=['Value'])
        obj_df.to_excel(writer, sheet_name='Objective_Function_Values')

In [11]:
# Define the scenarios with unique identifiers for each value
scenarios = {
    'S1.1': {'values': [80], 'param': 'k'},
    'S1.2': {'values': [120], 'param': 'k'},
    'S2.1': {'values': [60], 'param': 'C_bat'},
    'S2.2': {'values': [100], 'param': 'C_bat'},
    'S3.1': {'values': [5], 'param': 'alpha'},
    'S3.2': {'values': [15], 'param': 'alpha'},
    'S4.1': {'values': [5], 'param': 'beta'},
    'S4.2': {'values': [15], 'param': 'beta'},
    'S5.1': {'values': [0.20], 'param': 'P_sell'},
    'S5.2': {'values': [0.60], 'param': 'P_sell'}
}

In [12]:
# Loop through each scenario and parameter
for scenario_id, scenario in scenarios.items():
    param_name = scenario['param']
    
    # Initialize a list to collect results for the current scenario
    scenario_results = []
    
    for value in scenario['values']:
        kwargs = {
            'data': data,
            'pass_max': 5990,
            'mult_1': 1,
            'mult_2': 1,
            'k': 100,
            'alpha': 10,
            'beta': 7.2,
            'tee': False
        }
        kwargs[param_name] = value
        
        # Call your function with the updated parameters
        model = solve_opt(**kwargs)
    
        # Create a dynamic filename based on the scenario
        filename = os.path.join('sensitivity_results', f'output_energy_{scenario_id}.xlsx')

        # Save results to Excel
        save_to_excel(model, filename=filename)

        print(f"Results for {scenario_id} saved to {filename}")

Results for S1.1 saved to sensitivity_results/output_transport_S1.1.xlsx
Results for S1.3 saved to sensitivity_results/output_transport_S1.3.xlsx
Results for S2.2 saved to sensitivity_results/output_transport_S2.2.xlsx
Results for S2.3 saved to sensitivity_results/output_transport_S2.3.xlsx
Results for S3.1 saved to sensitivity_results/output_transport_S3.1.xlsx
Results for S3.3 saved to sensitivity_results/output_transport_S3.3.xlsx
Results for S4.2 saved to sensitivity_results/output_transport_S4.2.xlsx
Results for S4.3 saved to sensitivity_results/output_transport_S4.3.xlsx
Results for S5.1 saved to sensitivity_results/output_transport_S5.1.xlsx
Results for S5.3 saved to sensitivity_results/output_transport_S5.3.xlsx
